In [4]:
# =========================
# Cell 1 — ENV (MUST be first, before torch/transformers)
# =========================
from ft_pipeline.env import apply_env
apply_env()

import os
import gc
import torch

torch.backends.cuda.matmul.allow_tf32 = True
torch.set_float32_matmul_precision("high")

print("CUDA available:", torch.cuda.is_available())
print("CUDA device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else None)

print("allocated:", torch.cuda.memory_allocated()/1024**2, "MB")
print("reserved:",  torch.cuda.memory_reserved()/1024**2, "MB")

gc.collect()
torch.cuda.empty_cache()

print("allocated:", torch.cuda.memory_allocated()/1024**2, "MB")
print("reserved:",  torch.cuda.memory_reserved()/1024**2, "MB")


import logging
from ft_pipeline.logger import setup_logger
from ft_pipeline.config import FTConfig
from ft_pipeline.run_sft import run_finetune
from ft_pipeline.config import DPOCfg
from ft_pipeline.run_dpo import run_dpo




CUDA available: True
CUDA device: NVIDIA A100-SXM4-40GB
allocated: 9656.4775390625 MB
reserved: 35986.0 MB
allocated: 3875.18896484375 MB
reserved: 3944.0 MB


In [2]:
# =========================
# Cell — DPO Config
# =========================
# DPO (Direct Preference Optimization) after SFT (QLoRA LoRA-adapter)
# MamayLM-Gemma-3-12B (локальний шлях)
# GPU: A100 40GB, BF16, QLoRA 4-bit

from ft_pipeline.config import DPOCfg

cfg_dpo = DPOCfg(
    # ==========================================================
    # BASE MODEL + CONTINUATION FROM SFT
    # ==========================================================
    model_id="/home/jovyan/ai-models/MamayLM-Gemma-3-12B",               # path or HF repo id base model
    
    sft_adapter_dir="MamayLM-Gemma-3-12b_QLoRA_SFT/lora_adapter",           # LoRA-adapter after SFT,
    # sft_adapter_dir=None,                                              # if DPO from base
    
    dpo_train_jsonl="ft_datasets/dpo_train.jsonl",                      # train dataset in JSONL
    dpo_val_jsonl="ft_datasets/dpo_val.jsonl",                          # validation dataset in JSONL

    
    # out_dir="outputs_mamay12b_qlora_dpo",
    out_dir="MamayLM-Gemma-3-12b_QLoRA_SFT_DPO",
    # out_dir="MamayLM-Gemma-3-12b_QLoRA_DPO",
    
    # ==========================================================
    # SEQUENCE / BATCHING
    # ==========================================================
    max_seq_len=5000,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,  # effective batch = batch_size * grad_accum

    # ==========================================================
    # TRAINING SCHEDULE / OPTIM
    # ==========================================================
    learning_rate=0.0000005,     
    weight_decay=0.05,        # L2 regularization 
    num_train_epochs=2.0,     # (if  max_steps are provided - will ignored)
    max_steps=None,            # use instead - num_train_epochs
    warmup_ratio=0.05,
    lr_scheduler_type="cosine",  #scheduler: "cosine", "linear", ...
    logging_steps=5,
    eval_steps=50,
    save_steps=200,
    save_total_limit=2,

    # ==========================================================
    # DPO CORE
    # ==========================================================
    
    beta=0.03,   # to avoid overfit - safe start. -- if   accuracy getting to fast to 1.0 - change it  → 0.03

    use_bf16=True,            
    use_fp16=False,
    load_in_4bit=True,         # QLoRA (bitsandbytes 4-bit)
    attn_implementation="sdpa",   
    optim="paged_adamw_8bit",  # bitsandbytes to reduce the memory
    report_to="none",
    max_new_tokens_eval=512,  #  in A/B sanity (before/after)
)

print(cfg_dpo)

setup_logger(level=logging.INFO, log_file=f"{cfg_dpo.out_dir}/ft_run_dpo.log")

DPOCfg(model_id='/home/jovyan/ai-models/MamayLM-Gemma-3-12B', sft_adapter_dir='MamayLM-Gemma-3-12b_QLoRA_SFT/lora_adapter', dpo_train_jsonl='ft_datasets/dpo_train.jsonl', dpo_val_jsonl='ft_datasets/dpo_val.jsonl', out_dir='MamayLM-Gemma-3-12b_QLoRA_SFT_DPO', max_seq_len=5000, per_device_train_batch_size=1, per_device_eval_batch_size=1, gradient_accumulation_steps=8, learning_rate=5e-07, weight_decay=0.05, num_train_epochs=2.0, max_steps=None, warmup_ratio=0.05, lr_scheduler_type='cosine', logging_steps=5, eval_steps=50, save_steps=200, save_total_limit=2, beta=0.03, use_bf16=True, use_fp16=False, load_in_4bit=True, attn_implementation='sdpa', report_to='none', optim='paged_adamw_8bit', max_new_tokens_eval=512)


<Logger ft_pipeline (INFO)>

In [3]:


dpo_artifacts = run_dpo(
    cfg_dpo,
    do_ab_sanity=False,                 # optional control the json format - recommended 
    ab_val_jsonl="ft_datasets/sft_val.jsonl",   # if do_ab_sanity take the same from SFT
    ab_indices = list(range(15)),
    # ab_indices=[0, 1, 2, 10, 25, 50, 100, 150, 200, 250, 300],
    dataset_limits=(None, None),
)


08:49:15 | INFO    | === DPO RUN START ===
08:49:15 | INFO    | CUDA available=True
08:49:15 | INFO    | CUDA device=NVIDIA A100-SXM4-40GB
08:49:15 | INFO    | Loading tokenizer: /home/jovyan/ai-models/MamayLM-Gemma-3-12B
08:49:16 | INFO    | Tokenizer loaded
08:49:16 | INFO    | Loading DPO datasets
08:49:16 | INFO    |   train: ft_datasets/dpo_train.jsonl
08:49:16 | INFO    |   val:   ft_datasets/dpo_val.jsonl
08:49:41 | INFO    | DPO dataset ready | train=3094 | val=344
08:49:41 | INFO    | Loading base model (QLoRA)
08:49:41 | INFO    |   model_id: /home/jovyan/ai-models/MamayLM-Gemma-3-12B
08:49:41 | INFO    |   dtype: torch.bfloat16
08:49:41 | INFO    |   4bit: True
08:49:41 | INFO    |   attn_implementation: sdpa


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

08:51:06 | INFO    | Base model loaded
08:51:06 | INFO    | Enabling gradient checkpointing
08:51:06 | INFO    | Loading trainable LoRA adapter from: MamayLM-Gemma-3-12b_QLoRA_SFT/lora_adapter
08:51:08 | INFO    | Trainable adapter loaded
08:51:08 | INFO    | Enabled input require grads for gradient checkpointing
08:51:08 | INFO    | Trainable parameters:
trainable params: 68,456,448 || all params: 12,255,781,488 || trainable%: 0.5586
08:51:08 | INFO    | Building DPOConfig
08:51:08 | INFO    |   max_seq_len=5000
08:51:08 | INFO    |   beta=0.03
08:51:08 | INFO    |   lr=5e-07
08:51:08 | INFO    | Building DPOTrainer
08:51:08 | INFO    |   train_samples=3094
08:51:08 | INFO    |   val_samples=344


Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
Using auto half precision backend
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


08:51:09 | INFO    | Starting DPO training…


...

00:00:09 | INFO    | [step 450] eval_loss=0.0053 | gpu_mem(GB)=alloc:9.49 res:35.14 max_alloc:36.58 max_res:38.98 | elapsed=909.0m
00:00:09 | INFO    | DPO[EVAL] step=450
00:00:09 | INFO    | EarlyStop(metric=eval_loss): no improvement (current=0.005330 best=0.014001) | bad=2/2
00:00:09 | WARNING | EarlyStop(metric=eval_loss): STOP training (patience reached).




Training completed. Do not forget to share your model on huggingface.co/models =)




00:00:09 | INFO    | [step 450] tokens/sec=0.0 | gpu_mem(GB)=alloc:9.49 res:35.14 max_alloc:36.58 max_res:38.98 | elapsed=909.0m
00:00:09 | INFO    | DPO step=450
00:00:09 | INFO    | DPOMetricsCallback finished | csv=MamayLM-Gemma-3-12b_QLoRA_SFT_DPO/dpo_metrics.csv


chat template saved in MamayLM-Gemma-3-12b_QLoRA_SFT_DPO/tokenizer/chat_template.jinja
tokenizer config file saved in MamayLM-Gemma-3-12b_QLoRA_SFT_DPO/tokenizer/tokenizer_config.json
Special tokens file saved in MamayLM-Gemma-3-12b_QLoRA_SFT_DPO/tokenizer/special_tokens_map.json


00:00:17 | INFO    | DPO Artifacts: {'out_dir': 'MamayLM-Gemma-3-12b_QLoRA_SFT_DPO', 'lora_adapter_dir': 'MamayLM-Gemma-3-12b_QLoRA_SFT_DPO/lora_adapter', 'tokenizer_dir': 'MamayLM-Gemma-3-12b_QLoRA_SFT_DPO/tokenizer'}
00:00:17 | INFO    | === DPO RUN END ===


## 2️⃣ DPO — Direct Preference Optimization

### 🎯 Goal

Teach the model to **prefer the correct tariff**,  
when the format and style are already stable (after SFT).

---

### 📊 Key metrics (must-watch)

#### 🔹 `rewards/accuracies`
- normal range: **0.55–0.75**
- ❌ bad: **1.0 already at step 10–20**  
  (a sign of truncation or style mismatch)

#### 🔹 `rewards/margins`
- should be **> 0**
- good: grows slowly
- ❌ bad: quickly exceeds `10`

#### 🔹 `rewards/chosen > rewards/rejected`
- should be **always true**
- if not → DPO is not working

#### 🔹 `loss`
- does not have to go to 0
- ❌ `loss ≈ 0` + `accuracy = 1.0` → over-optimization

---

### 🎛️ Main knobs (DPO)

| Parameter | What it controls | When to change |
|---------|------------------|---------------|
| `beta` | preference strength | ↓ if accuracy quickly reaches 1.0 |
| `learning_rate` | adaptation speed | ↓ if margins “blow up” |
| `max_seq_len` | full context | ↑ if truncation occurs |
| `max_prompt_length` | prompt size | ↑ if `facts` are large |
| `max_steps` | overfitting control | ↓ if the model “wins” too fast |

---

### 🧠 Typical symptoms and fixes (DPO)

| Symptom | Likely cause | Fix |
|------|--------------|-----|
| accuracy = 1.0 from step 20 | truncated prompt | `max_seq_len = 4096` |
| margins > 10 | style mismatch | align chosen/rejected |
| broken JSON | beta too high | `beta = 0.03–0.05` |
| degradation on new cases | overfitting | ↓ LR, ↓ max_steps |

---

## 3️⃣ Recommended production order

1. **SFT**
   - stable JSON
   - correct language and structure
2. **DPO**
   - correct tariff selection
   - control via rewards/accuracies
3. **A/B sanity + business metrics**
   - tariff accuracy
   - JSON validity
   - `tariffId ∈ avail`

---

## ✅ Minimal checklist before “OK for prod”

- [ ] JSON parse rate = 100%
- [ ] `tariffId` is always valid
- [ ] DPO accuracy does not jump to 1.0 within 10–20 steps
- [ ] rewards/margins < ~8–10
- [ ] stability on unseen validation data

---

> 💡 Tip:  
> If DPO still feels “too easy” after fixing truncation —  
> the problem is almost always **style mismatch between chosen and rejected**,  
> not the hyperparameters.


## DPO metrics: what they mean & how to tune them (Tariff Recommender)

This notebook logs **DPOTrainer** metrics (pairwise preference learning).  
For each training row we have the same `prompt` and two answers:
- **chosen** = desired recommendation (e.g., matches real customer migration / label)
- **rejected** = plausible but undesired recommendation

DPO trains the model to make **chosen more likely than rejected** for the same prompt.

---

### Key logged metrics (what they mean)

#### 1) `loss`
- The DPO objective value. Lower is usually better *during* training.
- Use it mainly to detect instability (spikes/divergence).
- **Do not optimize loss alone** — you can overfit even when loss keeps decreasing.

**If loss is unstable / spikes:**
- Reduce `learning_rate`
- Increase `beta` **only if** accuracy is stuck ~0.5 (rare); otherwise keep beta small
- Consider gradient clipping (if available) and/or reduce batch noise (increase grad_accum)

---

#### 2) `rewards/accuracies`
- Fraction of pairs where:
  \[
  \log P(\text{chosen}|\text{prompt}) > \log P(\text{rejected}|\text{prompt})
  \]
- Interpretation:
  - `0.50` ≈ random preference
  - `0.70–0.85` = good learning
  - `>0.90` = strong preference alignment
  - `1.00` = model always prefers chosen (watch for over-optimization)

**If accuracy is low (≤0.6):**
- Check data quality (chosen/rejected consistency)
- Increase training steps slightly
- Increase `beta` a bit (e.g., `0.05 → 0.07`) **carefully**
- Ensure `max_prompt_length/max_length` prevents truncation (truncation kills preference signal)

**If accuracy hits 1.0 too fast:**
- You are likely in an easy regime → risk of overfitting the preference pairs
- Use fewer steps and/or smaller `learning_rate`
- Consider harder negatives (better rejected answers)

---

#### 3) `rewards/margins`
- Average preference margin:
  \[
  \text{margin} = \log P(\text{chosen}) - \log P(\text{rejected})
  \]
- Interpretation:
  - `~0` = model is unsure
  - `1–5` = healthy confidence
  - `>10` = very confident
  - `>>10` (e.g., 15–20+) = can indicate **over-optimization** (model becomes too “certain”)

**If margins grow very large early:**
- Reduce `max_steps` / epochs
- Reduce `learning_rate` (e.g., `1e-5 → 5e-6`)
- Reduce `beta` (makes updates less aggressive)

**If margins stay near 0:**
- Increase steps slightly
- Slightly increase `beta` (e.g., `0.03 → 0.05`)
- Verify your rejected answers are meaningfully different (hard negatives help)

---

#### 4) `rewards/chosen` and `rewards/rejected`
- These are model “scores” for each side (often derived from log-probs).
- What you want to see:
  - `chosen` trending up
  - `rejected` trending down
  - Their difference aligns with `rewards/margins`

**If both go up together:**
- You may be optimizing style rather than preference separation
- Consider stronger negatives / improve rejected construction
- Ensure loss is computed on the full response tokens (no masking bug)

---

#### 5) `learning_rate`
- The current LR. If logs show `0.0000`, your logger may be reading the wrong key.
- Make sure logging normalizes:
  - `learning_rate` OR `lr` → log as `learning_rate`

---

### What to tune (control knobs)

#### A) `learning_rate` (most important)
- **DPO should use smaller LR than SFT.**
- Safe defaults:
  - `1e-5` (`0.00001`) for DPO
  - `5e-6` (`0.000005`) if margins explode or accuracy hits 1.0 too fast

Symptoms → Fix:
- margins jump fast / accuracy → 1.0 quickly → **lower LR**
- loss noisy / spikes → **lower LR**
- no learning (accuracy ~0.5) → slightly higher LR or more steps (but first check data)

---

#### B) `beta` (aggressiveness of preference push)
- Think of `beta` as how strongly we force chosen > rejected.
- Safe starting point: `beta = 0.05`

Symptoms → Fix:
- accuracy stuck low → increase `beta` a bit (e.g., `0.05 → 0.07`)
- margins explode / overconfident → reduce `beta` (e.g., `0.05 → 0.03`)

---

#### C) `max_steps` / epochs (how long you train)
- DPO often converges fast.
- Watch for:
  - accuracy near 1.0 + margins rising fast → stop early

Practical guidance:
- Start with **200–400 steps** total (for ~3k rows with grad_accum=8 this is often enough)
- Use early stopping on `eval_loss` and/or a custom rule on margins/accuracy if available.

---

#### D) `max_length` / `max_prompt_length` (avoid truncation)
If prompt p95 ~3000 and answers ~300–400 tokens:
- Use `max_length = 4096`
- Use `max_prompt_length ~ 3500–3600` (leave room for response)

Symptoms of truncation:
- accuracy fails to improve
- margins stay near 0
- training becomes noisy

---

### What to watch for in practice (quick checklist)

✅ Healthy DPO run:
- `rewards/accuracies` rises from ~0.5 → 0.8–0.95
- `rewards/margins` increases to a moderate range (often 2–10)
- `chosen` up, `rejected` down
- eval metrics stabilize without degrading output format (JSON validity)

⚠️ Over-optimization signs:
- `rewards/accuracies` → 1.0 very early
- `rewards/margins` keeps climbing to very high values (e.g., >10–15) while eval stops improving
- output quality issues (e.g., worse JSON compliance or less robust behavior)

**Fix over-optimization:**
- fewer steps
- lower `learning_rate`
- lower `beta`

---

### Optional: add a format metric (recommended for production)
Besides DPO metrics, track a **task-format KPI** on a fixed subset (A/B sanity):
- `valid_json_rate`
- tariff field presence: `tariffId`, `templateId`, etc.

If `valid_json_rate` drops while DPO metrics “improve”:
- prefer stopping early / reduce aggressiveness
- keep generation deterministic during evaluation (`do_sample=False`)
